In [6]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import time
import cPickle as pickle
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
from sklearn.metrics import coverage_error
import sklearn.metrics
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import logging

In [7]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [8]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [9]:
STOP_WORDS = nltk.corpus.stopwords.words('english')
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"
MIN_WORD_COUNT = 5
MIN_SIZE = 0
NUM_CORES = 7

In [10]:
SAMPLE_RATIO = 0.006

In [11]:
#training_file = "/home/local/shalaby/docs_output_sample_100.json"
training_file = "/home/local/shalaby/docs_output.json"
doc_classifications_map_file = "/home/local/shalaby/doc_classification_map.pkl"
sections_file = "/home/local/shalaby/sections.pkl"
training_docs_list_file = "/home/local/shalaby/training_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
validation_docs_list_file = "/home/local/shalaby/validation_docs_list.pkl"

In [12]:
model_save_location = "/home/local/shalaby/models/scikit-svm/sample_" + str(SAMPLE_RATIO) + "/"

In [13]:
%%time
doc_classifications_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
#open(sections_file).read()

CPU times: user 27.4 s, sys: 1min 6s, total: 1min 34s
Wall time: 1min 34s


In [14]:
len(training_docs_list)

49544

In [15]:
len(validation_docs_list)

321473

In [16]:
def stemtokenizer(text):
    """ MAIN FUNCTION to get clean stems out of a text. A list of clean stems are returned """
    tokenizer = RegexpTokenizer(r'\s+', gaps=True)
    tokens = tokenizer.tokenize(text)
    stems = []  # result
    for token in tokens:
        stem = token.lower()
        stem = stem.strip(string.punctuation)
        if stem:
            if is_number(stem):
                stem = NUMBER_INDICATOR
            elif is_currency(stem):
                stem = CURRENCY_INDICATOR
            elif is_chemical(stem):
                stem = CHEMICAL_INDICATOR
            elif is_stopword(stem):
                stem = None
            else:
                stem = stem.strip(string.punctuation)
            if stem and len(stem) >= MIN_SIZE:
                # extract uni-grams
                stems.append(stem)
    del tokens
    return stems

def is_stopword(word):
  return word in STOP_WORDS

def is_number(str):
    """ Returns true if given string is a number (float or int)"""
    try:
        float(str.replace(",", ""))
        return True
    except ValueError:
        return False

def is_currency(str):
    return str[0] == "$"

def is_chemical(str):
    return str.count("-") > 3

### Doc2vec and SVM Parameters

In [22]:
DOC2VEC_SIZE = 400
DOC2VEC_WINDOW = 8
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-5
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 0
DOC2VEC_EPOCHS = 1
DOC2VEC_MAX_EPOCHS = 20
REPORT_DELAY = 30 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 1000 # report the progress every x terms

In [18]:
SVM_ITERATIONS = 1000
SVM_CONVERGENCE = 0.001
SVM_REG = 1/0.1 # scikit uses a C parameter not a lambda

In [19]:
class LabeledLineSentence(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for line in open(self.filename):
            (doc_id, text) = eval(line)
            if doc_id in training_docs_list:
                yield LabeledSentence(words=stemtokenizer(text), tags=[doc_id])

## Create the Doc2vec model

In [20]:
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE , window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT, 
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=0,
                iter=DOC2VEC_EPOCHS)

In [23]:
%%time
doc2vec_model.build_vocab(sentences=LabeledLineSentence(training_file), progress_per=REPORT_VOCAB_PROGRESS)

2016-09-01 01:38:02,198 : INFO : collecting all words and their counts
2016-09-01 01:38:02,337 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2016-09-01 01:42:31,289 : INFO : PROGRESS: at example #1000, processed 5129048 words (19070/s), 139924 word types, 1000 tags
2016-09-01 01:46:53,692 : INFO : PROGRESS: at example #2000, processed 9805080 words (17820/s), 227427 word types, 2000 tags
2016-09-01 01:51:20,712 : INFO : PROGRESS: at example #3000, processed 14569369 words (17842/s), 290517 word types, 3000 tags
2016-09-01 01:55:36,396 : INFO : PROGRESS: at example #4000, processed 19027847 words (17437/s), 343413 word types, 4000 tags
2016-09-01 02:00:02,483 : INFO : PROGRESS: at example #5000, processed 24491779 words (20534/s), 506953 word types, 5000 tags
2016-09-01 02:04:28,990 : INFO : PROGRESS: at example #6000, processed 29393944 words (18394/s), 572595 word types, 6000 tags
2016-09-01 02:08:47,034 : INFO : PROGRESS: at example #7000, processed 

CPU times: user 3h 35min 44s, sys: 5min 3s, total: 3h 40min 47s
Wall time: 3h 41min 52s


In [24]:
%%time
DOC2VEC_MAX_EPOCHS = 1
doc2vec_model.min_alpha = 0.025
for epoch in range(DOC2VEC_MAX_EPOCHS):
    doc2vec_model.train(sentences=LabeledLineSentence(training_file), report_delay=REPORT_DELAY)
    doc2vec_model.alpha -= 0.001  # decrease the learning rate
    doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay

2016-09-01 09:59:07,251 : INFO : training model with 7 workers on 412602 vocabulary and 400 features, using sg=0 hs=0 sample=1e-05 negative=10
2016-09-01 09:59:07,253 : INFO : expecting 49544 sentences, matching count from corpus used for vocabulary survey
2016-09-01 09:59:14,954 : INFO : PROGRESS: at 0.00% examples, 175 words/s, in_qsize 5, out_qsize 0
2016-09-01 09:59:45,674 : INFO : PROGRESS: at 0.22% examples, 5039 words/s, in_qsize 0, out_qsize 0
2016-09-01 10:00:15,696 : INFO : PROGRESS: at 0.44% examples, 5682 words/s, in_qsize 0, out_qsize 0
2016-09-01 10:00:46,135 : INFO : PROGRESS: at 0.65% examples, 5660 words/s, in_qsize 0, out_qsize 0
2016-09-01 10:01:17,187 : INFO : PROGRESS: at 0.87% examples, 5895 words/s, in_qsize 0, out_qsize 0
2016-09-01 10:01:47,403 : INFO : PROGRESS: at 1.11% examples, 6013 words/s, in_qsize 0, out_qsize 0
2016-09-01 10:02:18,318 : INFO : PROGRESS: at 1.30% examples, 6039 words/s, in_qsize 0, out_qsize 0
2016-09-01 10:02:49,311 : INFO : PROGRESS: a

NameError: name 'model' is not defined

In [25]:
doc2vec_model.min_alpha = doc2vec_model.alpha

### Load Doc2vec Model

In [12]:
%%time
file_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_hs_{}_iter_{}'.format(DOC2VEC_SIZE, DOC2VEC_WINDOW, 
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE)

doc2vec_model = Doc2Vec.load('/home/local/shalaby/models/{}'.format(file_name), mmap='r')

2016-08-23 03:34:43,130 : INFO : loading Doc2Vec object from /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10
2016-08-23 03:39:37,655 : INFO : loading docvecs recursively from /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.docvecs.* with mmap=r
2016-08-23 03:39:37,657 : INFO : loading doctag_syn0 from /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.docvecs.doctag_syn0.npy with mmap=r
2016-08-23 03:39:37,659 : INFO : loading syn1neg from /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.syn1neg.npy with mmap=r
2016-08-23 03:39:37,715 : INFO : loading syn0 from /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.syn0.npy with mmap=r
2016-08-23 03:39:55,534 : INFO : setting ignored attribute syn0norm to None
2016-08-23 03:39:55,535 : INFO : setting ignored attribute cum_table to None


CPU times: user 1min 3s, sys: 4min 24s, total: 5min 27s
Wall time: 5min 45s


In [13]:
doc2vec_model

In [21]:
doc2vec_model.docvecs

## Actual Training

In [14]:
MultiLabelBinarizer().fit_transform([["B","C"],["D"],["E"],["A"]])

array([[0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [15]:
training_vectors = []
training_labels = []
for doc_id in training_docs_list:
    training_vectors.append(doc2vec_model.docvecs[doc_id])
    training_labels.append([classf for classf in doc_classifications_map[doc_id] if classf in sections])
training_vectors = np.array(training_vectors)

In [16]:
binary_labels = MultiLabelBinarizer().fit_transform(training_labels)

In [17]:
binary_labels.shape

(1286325, 8)

In [18]:
training_vectors.shape

(1286325, 400)

In [19]:
svm_model = OneVsRestClassifier(LinearSVC(penalty='l2', tol=SVM_CONVERGENCE, 
                                          C=SVM_REG, verbose=1, random_state=SVM_SEED, max_iter=SVM_ITERATIONS))

In [20]:
%%time
svm_model.fit(training_vectors, binary_labels)
pickle.dump(svm_model, open(model_save_location + file_name + "_mmap", 'w'))

[LibLinear]

/home/stud/shalaby/.virtualenv/thesis-env/lib/python2.7/site-packages/sklearn/svm/base.py:924: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]CPU times: user 6h 10min 21s, sys: 1h 10min 58s, total: 7h 21min 19s
Wall time: 7h 21min 1s


## Inference Step for Validation vectors

In [27]:
class ValidationDocumentGenerator(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for line in open(self.filename):
            (doc_id, text) = eval(line)
            if doc_id in validation_docs_list:
                yield doc_id, stemtokenizer(text)

In [ ]:
%%time
validation_documents_reps = {}
i = 0
for (doc_id, doc_contents_array) in ValidationDocumentGenerator(training_file):
    i += 1
    if i % 1000 == 0: print i
    validation_documents_reps[doc_id] = doc2vec_model.infer_vector(doc_contents_array)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [38]:
print "lksjdflsjkdf"

lksjdflsjkdf


In [39]:
%%time
validation_vectors = []
validation_labels = []
for validation_doc_id in validation_docs_list:
    validation_vectors.append(validation_documents_reps[validation_doc_id])
    validation_labels.append([classf for classf in doc_classifications_map[validation_doc_id] if classf in sections])
validation_vectors = np.array(validation_vectors)

CPU times: user 1.93 s, sys: 1min 17s, total: 1min 19s
Wall time: 1min 19s


In [40]:
validation_vectors.shape

(321473, 400)

In [42]:
pickle.dump(validation_vectors, open('/home/local/shalaby/validation_data/' + file_name + "_validation_vectors",'w'))
pickle.dump(validation_labels, open('/home/local/shalaby/validation_data/' + file_name + "_validation_labels",'w'))

In [ ]:
validation_binary_labels = MultiLabelBinarizer().fit_transform(training_labels)

In [ ]:
%%time
validation_prediction = svm_model.predict(validation_vectors)

### Increase the number of iterations

In [ ]:
svm_model2 = OneVsRestClassifier(LinearSVC(penalty='l2', tol=SVM_CONVERGENCE, 
                                          C=SVM_REG, verbose=1, random_state=SVM_SEED, max_iter=10000))
svm_model2.fit(training_vectors, binary_labels)

[LibLinear][LibLinear]

### Create the model

In [34]:
print model

Doc2Vec(dm/c,d400,n10,w8,mc5,s1e-05,t7)


#### Initialize the model using the vocab from the previously trained model

In [117]:
model.reset_from(loaded_model)

2016-08-16 15:49:28,921 : INFO : using concatenative 6800-dimensional layer1
using concatenative 6800-dimensional layer1
2016-08-16 15:49:28,925 : INFO : resetting layer weights
resetting layer weights


#### Now for the actual training

In [ ]:
%%time
model.build_vocab(sentences=LabeledLineSentence(training_file), progress_per=REPORT_VOCAB_PROGRESS)

2016-08-21 05:36:43,143 : INFO : collecting all words and their counts
2016-08-21 05:36:43,576 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


In [25]:
%%time
model.train(sentences=LabeledLineSentence(training_file), report_delay=REPORT_DELAY)

2016-08-21 05:32:48,403 : INFO : training model with 7 workers on 5794 vocabulary and 6800 features, using sg=0 hs=0 sample=1e-05 negative=10
2016-08-21 05:32:48,405 : INFO : expecting 65 sentences, matching count from corpus used for vocabulary survey
2016-08-21 05:32:51,066 : INFO : PROGRESS: at 1.54% examples, 658 words/s, in_qsize 0, out_qsize 0
2016-08-21 05:33:05,759 : INFO : worker thread finished; awaiting finish of 6 more threads
2016-08-21 05:33:05,762 : INFO : worker thread finished; awaiting finish of 5 more threads
2016-08-21 05:33:05,763 : INFO : worker thread finished; awaiting finish of 4 more threads
2016-08-21 05:33:05,865 : INFO : worker thread finished; awaiting finish of 3 more threads
2016-08-21 05:33:05,894 : INFO : worker thread finished; awaiting finish of 2 more threads
2016-08-21 05:33:06,142 : INFO : worker thread finished; awaiting finish of 1 more threads
2016-08-21 05:33:06,551 : INFO : worker thread finished; awaiting finish of 0 more threads
2016-08-21 

CPU times: user 32.1 s, sys: 2.58 s, total: 34.7 s
Wall time: 18.2 s


70839

In [134]:
# file_name = 'doc2vec_size_{}_w_{}_type_{}_hs_{}_iter_{}'.format(DOC2VEC_SIZE, DOC2VEC_WINDOW, 
#                                                                 'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
#                                                                 DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE)
file_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_hs_{}_neg_{}_vocabsize_{}_iter_{}'.format(DOC2VEC_SIZE, DOC2VEC_WINDOW, 
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE), DOC2VEC_EPOCHS)
model.save('/home/local/shalaby/models/{}'.format(file_name))

2016-08-20 15:22:45,778 : INFO : saving Doc2Vec object under /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10, separately None
saving Doc2Vec object under /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10, separately None
2016-08-20 15:22:45,783 : INFO : storing numpy array 'doctag_syn0' to /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.docvecs.doctag_syn0.npy
storing numpy array 'doctag_syn0' to /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.docvecs.doctag_syn0.npy
2016-08-20 15:22:50,011 : INFO : storing numpy array 'syn1neg' to /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.syn1neg.npy
storing numpy array 'syn1neg' to /home/local/shalaby/models/doc2vec_size_400_w_8_type_dm_concat_1_mean_0_hs_0_iter_10.syn1neg.npy
2016-08-20 15:40:39,540 : INFO : not storing attribute syn0norm
not storing attribute syn

In [76]:
model.most_similar('08887671')

2016-08-16 14:34:05,764 : INFO : precomputing L2-norms of word weight vectors
precomputing L2-norms of word weight vectors


AttributeError: 'Doc2Vec' object has no attribute 'syn0'

In [77]:
model['08887671']

AttributeError: 'Doc2Vec' object has no attribute 'syn0'

In [116]:
len(loaded_model.vocab)

6147817

In [99]:
from collections import defaultdict
model.raw_vocab = defaultdict(int)

In [100]:
model.raw_vocab

defaultdict(int, {})

In [83]:
dir(model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__ignoreds',
 '__init__',
 '__module__',
 '__new__',
 '__numpys',
 '__recursive_saveloads',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__scipys',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_do_train_job',
 '_load_specials',
 '_raw_word_count',
 '_save_specials',
 '_smart_save',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'cbow_mean',
 'clear_sims',
 'comment',
 'corpus_count',
 'create_binary_tree',
 'dbow',
 'dbow_words',
 'dm',
 'dm_concat',
 'dm_tag_count',
 'docvecs',
 'doesnt_match',
 'estimate_memory',
 'finalize_vocab',
 'hashfxn',
 'hs',
 'index2word',
 'infer_vector',
 'init_sims',
 'intersect_word2vec_format',
 'iter',
 'layer1_size',
 'load',
 'load_word2vec_format',
 'log_accuracy',
 'make_cum_table',
 'max_vocab_size',
 'min_alpha',
 'min_alph

In [95]:
len(model.vocab)

6147817

In [96]:
model.wmdistance

<bound method Doc2Vec.wmdistance of <gensim.models.doc2vec.Doc2Vec object at 0x7fd85f134210>>